In [97]:
import torch
import numpy as np
import pandas as pd

In [147]:
df = pd.read_csv("dataset/framingham.csv")
df.info()
df=df.fillna((df.min()+df.max())/2)
n=len(df)
df=df.T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             4238 non-null   int64  
 1   age              4238 non-null   int64  
 2   education        4133 non-null   float64
 3   currentSmoker    4238 non-null   int64  
 4   cigsPerDay       4209 non-null   float64
 5   BPMeds           4185 non-null   float64
 6   prevalentStroke  4238 non-null   int64  
 7   prevalentHyp     4238 non-null   int64  
 8   diabetes         4238 non-null   int64  
 9   totChol          4188 non-null   float64
 10  sysBP            4238 non-null   float64
 11  diaBP            4238 non-null   float64
 12  BMI              4219 non-null   float64
 13  heartRate        4237 non-null   float64
 14  glucose          3850 non-null   float64
 15  TenYearCHD       4238 non-null   int64  
dtypes: float64(9), int64(7)
memory usage: 529.9 KB


In [148]:
def sigmoid(z):
    return 1/(1+np.exp(z))

In [149]:
def ReLU(z):
    return np.maximum(0,z)

In [150]:
def tanh(z):
    return (np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))

In [151]:
def LeakyReLU(z):
    return np.maximum(0.01*z,z)

In [152]:
y=df.iloc[15,:].values.reshape(1,-1)
x=df.iloc[:15,:].values.reshape(15,-15)
print (x)

[[  1.     0.     1.   ...   0.     0.     0.  ]
 [ 39.    46.    48.   ...  48.    44.    52.  ]
 [  4.     2.     1.   ...   2.     1.     2.  ]
 ...
 [ 26.97  28.73  25.34 ...  22.    19.16  21.47]
 [ 80.    95.    75.   ...  84.    86.    80.  ]
 [ 77.    76.    70.   ...  86.   217.   107.  ]]


In [1]:
def norm(x):
    x=x-(np.amax(x,axis=1,keepdims=True)+np.amin(x,axis=1,keepdims=True))/2
    m=np.sum(x**2,axis=1,keepdims=True)
    m=np.sqrt(m)
    x=x/m
    return x

In [154]:
print(x.shape)

n=(int)(n*0.8)
print (n)

print (df.shape)

(15, 4238)
3390
(16, 4238)


In [155]:
x_train=df.iloc[:15,:n].values.reshape(15,-1)
x_train=norm(x_train)
y_train=df.iloc[15,:n].values.reshape(1,-1)
x_test=df.iloc[:15,n:].values.reshape(15,-1)
x_test=norm(x_test)
y_test=df.iloc[15,n:].values.reshape(1,-1)

In [156]:
print (x_train.shape
      )

(15, 3390)


In [171]:
#init
w1=(np.random.rand(4,15)-1)
w2=(np.random.rand(5,4)-1)
w3=(np.random.rand(1,5)-1)
#layers=2 
#layer hidden has 2 nodes
#layer output has 1 nodes
b1=np.zeros((4,1))
b2=np.zeros((5,1))
b3=np.zeros((1,1))


In [172]:
lr=0.01
kr=0.8
lambd=0.4
eg=np.random.rand(w1.shape[0],w1.shape[1])<kr
print (eg)
for i in range(100):
    #forward
    z1=np.dot(w1*(np.random.rand(w1.shape[0],w1.shape[1])<kr),x_train)+b1
    norm(z1)
    a1=z1
    z2=np.dot(w2*(np.random.rand(w2.shape[0],w2.shape[1])<kr),a1)+b2
    norm(z2)
    a2=ReLU(z2)
    z3=np.dot(w3*(np.random.rand(w3.shape[0],w3.shape[1])<kr),a2)+b3
    norm(z3)
    a3=sigmoid(z3)
    
    #cal loss
    #backward
    dz3=a3-y_train
    dw3=1/n*np.dot(dz3,a2.T)+w3*lambd/n
    db3=1/n*np.sum(dz3,axis=1,keepdims=True)
    dz2=1/n*np.dot(w3.T,dz3)*(z2>0)
    dw2=1/n*np.dot(dz2,a1.T)+w2*lambd/n
    db2=1/n*np.sum(dz2,axis=1,keepdims=True)
    dz1=1/n*np.dot(w2.T,dz2)
    dw1=1/n*np.dot(dz1,x_train.T)+w1*lambd/n
    db1=1/n*np.sum(dz1,axis=1,keepdims=True)
    #decay
    dw1=
    #update 
    w3=w3-lr*dw3
    b3=b3-lr*db3
    w2=w2-lr*dw2
    b2=b2-lr*db2
    w1=w1-lr*dw1
    b1=b1-lr*db1
    #drop out
    z1_pred=np.dot(w1,x_test)+b1
    a1_pred=z1_pred
    z2_pred=np.dot(w2,a1_pred)+b2
    a2_pred=ReLU(z2_pred)
    z3_pred=np.dot(w3,a2_pred)+b3
    a3_pred=sigmoid(z3_pred)
    y_pred=np.round(a3_pred)

[[False  True  True  True  True  True  True False  True  True  True  True
   True  True  True]
 [ True  True False False  True  True  True False  True  True  True False
   True  True  True]
 [ True False  True  True  True  True False  True False False False  True
   True  True  True]
 [False  True  True False  True  True  True  True  True  True False  True
   True False  True]]


In [173]:
#evaluate

z1_pred=np.dot(w1,x_test)+b1
a1_pred=z1_pred
z2_pred=np.dot(w2,a1_pred)+b2
a2_pred=ReLU(z2_pred)
z3_pred=np.dot(w3,a2_pred)+b3
a3_pred=sigmoid(z3_pred)
y_pred=np.round(a3_pred)
print(y_test-a3_pred)

print(w1)
print(w2)
res=abs(y_test-y_pred)
count =(1-((np.sum(res))/len(res.T)))*100
print("correct prediction:",count,"%")

[[-0.59723093 -0.59723093 -0.59723093 -0.59723093 -0.59723093 -0.59723093
  -0.59723093 -0.59723093 -0.59723093 -0.59723093 -0.59723093 -0.59723093
   0.40276907 -0.59723093 -0.59723093 -0.59723093  0.40276907  0.40276907
  -0.59723093 -0.59723093 -0.59723093 -0.59723093 -0.59723093 -0.59723093
  -0.59723093 -0.59723093 -0.59723093 -0.59723093  0.40276907 -0.59723093
  -0.59723093 -0.59723093 -0.59723093 -0.59723093 -0.59723093 -0.59723093
  -0.59723093 -0.59723093 -0.59723093 -0.59723093 -0.59723093 -0.59723093
  -0.59723093 -0.59723093 -0.59723093 -0.59723093 -0.59723093 -0.59723093
  -0.59723093  0.40276907 -0.59723093 -0.59723093 -0.59723093 -0.59723093
  -0.59723093  0.40276907  0.40276907 -0.59723093 -0.59723093  0.39977941
  -0.59723093 -0.59723093 -0.59723093 -0.59723093 -0.59723093 -0.59723093
   0.40276907 -0.59723093 -0.59723093 -0.59723093 -0.59723093 -0.59723093
   0.40276907 -0.59723093 -0.59723093 -0.59723093 -0.59723093 -0.59723093
  -0.59723093 -0.59723093 -0.59723093 